In [ ]:

import os
import cv2     as cv
import numpy   as np
import calibri as cal




class Data_Collector:

    def __init__(self,imgs_dir_path, cam_calibration_path):
        self.imgs_dir_path = imgs_dir_path
        data               = np.load(cam_calibration_path,allow_pickle=True)
        self.mtx           = data['mtx']
        self.dist          = data['dist']
    
    def take_image(self, img_name , cam_idx):

        abs_path = os.path.join(self.imgs_dir_path,img_name)

        cap = cv.VideoCapture(cam_idx)
        
        if not cap.isOpened():
            print("Cannot open camera")
            exit()

        ret, frame = cap.read()

        if not ret:
            print("Can't take an image. Exiting ....")
            cap.release()
            cv.destroyAllWindows()
            exit()
        else:       
            img = np.asarray(frame)
            undistored_img = cal.undistort(img,self.mtx,self.dist)
            success = cv.imwrite(abs_path,undistored_img)
            
            if success:
                print(f"Image successfully saved to {abs_path}")
            else:
                print(f"An error occured. The image was not able to be captured.")
        cap.release()
        cv.destroyAllWindows()

    
    def get_img_count(self, filepath):
        
        img_dir = os.listdir(filepath)
        
        count = 0

        for file in img_dir:
            if file.endswith(".jpg") or file.endswith(".png") or file.endswith(".jpeg"):
                count+=1

        return count

    def find_camera_idx():
        
        for cam_idx in range(-1,10):

            cap = cv.VideoCapture(cam_idx)
            
            if cap.isOpened():
                print(f"Camera is open on idx {cam_idx}")


        




In [4]:
obj = Data_Collector()

In [ ]:
obj.take_image()